In [139]:
import pandas as pd
from csv import reader
import pprint
from collections import defaultdict


In [140]:
# load the data

In [141]:
squad_list = pd.read_csv('C:/Users/jw156/Ironhack/football_teammates/Footballers/new_jonty/football_squad.csv', index_col=0)

In [142]:
squad_list

,Player,Club,Year,Player ID,Team ID,Appearances
0,Tony Adams,arsenal,1993,31,arsenal_1993,33
1,Steve Bould,arsenal,1993,484,arsenal_1993,24
2,Kevin Campbell,arsenal,1993,196,arsenal_1993,32
3,Jimmy Carter,arsenal,1993,30111,arsenal_1993,11
4,Paul Davis,arsenal,1993,27871,arsenal_1993,6
...,...,...,...,...,...,...
15807,Rui Patrício,wolverhampton-wanderers,2021,228012,wolverhampton-wanderers_2021,29
15808,Romain Saïss,wolverhampton-wanderers,2021,252857,wolverhampton-wanderers_2021,20
15809,Adama Traoré,wolverhampton-wanderers,2021,241740,wolverhampton-wanderers_2021,20
15810,Vitinha,wolverhampton-wanderers,2021,265842,wolverhampton-wanderers_2021,2


In [143]:
# dataframe finding shared players

In [144]:
# find reserves and first-time

In [145]:
def player_or_squadman(appearances):
    if appearances == 0:
        return 'squad_player'
    else:
        return 'first_team_player'

In [146]:
squad_list['reserve_or_player'] = squad_list['Appearances'].apply(player_or_squadman)

#  Shared Players 'inc. reserves'

In [167]:
def shared_players(player1= '', player2='', reserve=True, squad_list=squad_list):
    all_player_id_to_team_ids_map = defaultdict(list)
    team_id_to_player_id = defaultdict(list)
    player_id_to_player_name = {}
    player_name_to_player_id_map = {}

    if reserve == False:
        squad_list = squad_list[squad_list['reserve_or_player']=='first_team_player']
    else:
        squad_list = squad_list

    players = squad_list.Player.tolist()
    all_player_id = squad_list['Player ID'].tolist()
    team_id = squad_list['Team ID'].tolist()

    for p, p_id, t_id in zip(players, all_player_id, team_id):
        player_name_to_player_id_map[p] = p_id
        all_player_id_to_team_ids_map[p_id].append(t_id)
        team_id_to_player_id[t_id].append(p_id)
        player_id_to_player_name[p_id] = p


    player1 = player_name_to_player_id_map[player1]
    player_1_team_ids = all_player_id_to_team_ids_map[player1]

    all_players_1 = set([])
    for team in player_1_team_ids:
        list_squads = team_id_to_player_id[team]
        for player in list_squads:
            all_players_1.add(player)
    player2 = player_name_to_player_id_map[player2]
    player_2_team_ids = all_player_id_to_team_ids_map[player2]

    all_players_2 = set([])
    for team in player_2_team_ids:
        list_squads = team_id_to_player_id[team]
        for player in list_squads:
            all_players_2.add(player)

    shared_players = list(all_players_1.intersection(all_players_2))
    shared_name_players = [player_id_to_player_name[player] for player in shared_players]
    return shared_name_players

In [172]:
shared_players('Gary Neville', 'Steven Gerrard', reserve=False)

['Michael Owen', 'Paul Ince']